## 👨🏻‍💻 Stock Technical Analysis Framework

### 📥 Setups

Installation and Import of required packages

In [2]:
# !pip install -r requirements.txt

In [3]:
import yfinance as yf

import pandas as pd

Initialization of Analysis Parameters

In [4]:
TICKER: str = 'AAPL'

START_DATE: str = '2019-01-01'
END_DATE: str = '2025-07-25'

### 🏗️ Data Acquisition

Function for fetching stock OHCLV data

In [5]:
def fetch_price_data(ticker: str, start: str = START_DATE, end: str = END_DATE) -> pd.DataFrame:
    """
    
    """
    
    data = yf.download(ticker, start=start, end=end, auto_adjust=True)
    data.columns = ['Close', 'High', 'Low', 'Open', 'Volume']

    return data

The indicators will be split into 4 catgories, volume, volatility, momentum, and trend. The list of specific indicators in each is as follows.
- Volume: 
| Provider            |  Pros                                                                                      | Cons                                                                                       |
|:-------------------:|:------------------------------------------------------------------------------------------:|:------------------------------------------------------------------------------------------:|
| Alpha Vantage       | Awesome all-around -- Almost complete data, Great documentation, Simple Python SDK; NASDAQ licensed | Restrained service at free plan -- Low rate limits, No adjusted close price, No realtime US market data |
- Volatility:
- Momentum:
- Trend: